In [1]:
from tkinter import *
from tkinter.ttk import *
import tkinter.font as font
from tkinter import filedialog
import os
from PIL import ImageTk, Image
import tensorflow as tf
import cv2
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
from numpy import argmax
from keras.models import load_model
import PIL.Image
import tkinter as tk

In [2]:
import warnings
warnings.filterwarnings(action='ignore')
import re
import pymongo
import pandas as pd
from konlpy.tag import Okt
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import datasets, layers, models, optimizers

In [3]:
def classify_cloud(img): # 구름 유형 분류하기
    # 라이브러리 추가
    import os.path
    import cv2
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import tensorflow as tf
    import seaborn as sns
    import PIL
    import pathlib
    
    from glob import glob
    from tensorflow.keras import layers
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report,accuracy_score
    from keras.models import load_model
    
    # 파일 경로 지정
    path = "C:/Users/hi/Desktop/DeepLearning/data/chatbot_dataset/cloud_data" # 분석하고자 하는 사진이 있는 폴더 경로
    change_dir = os.chdir(path)
    test_dir = pathlib.Path(os.getcwd())
    
    # 분석하고자 하는 이미지 선택
    real_img = list(test_dir.glob(img))
    
    # 이미지 출력
    PIL.Image.open(str(real_img[0]))
    
    # 이미지 크기 지정
    img_height = 180
    img_width = 180
    
    # 분류 클래스 이름 지정
    class_names = ['고적운(altocumulus, Ac)', '고층운(altostratus, As)', '적란운(cumulonimbus, Cb)',
               '권적운(cirrocumulus, Cc)', '권운(cirrus, Ci)', '권층운(cirrostratus, Cs)', 
               '비행운(contrail, Ct)', '적운(cumulus, Cu)', '난층운(nimbostratus, Ns)',
               '층적운(stratocumulus, Sc)', '충운(stratus, St)'
               ]
    
    # 이미지 전처리
    img = keras.preprocessing.image.load_img(
        real_img[0], target_size=(img_height, img_width)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    # 구름 유형 분류 모델 불러오기
    model2 = load_model(path + '/cloud_model.h5')

    # 모델 예측
    predictions = model2.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    # 구름 유형 출력
    chat_bot = "runway: 사진 속 구름은 {}이네요!!".format(class_names[np.argmax(score)])
    Output.insert(END,chat_bot + '\n')
    # 예측한 구름 유형 반환
    cloud = class_names[np.argmax(score)]
    return cloud

In [4]:
def search_road_addr():
    # 네이버 openAPI 검색 > 지역 활용
    # 키워드를 통해 도로명 주소를 얻음
    
    import urllib.request
    import json
    from urllib import parse
    
    #애플리케이션 클라이언트 id 및 secret
    client_id = "Your_naver_developers_client_id" # 네이버 developers에서 발급받은 client_id
    client_secret = "Your_naver_developers_client_secret" # 네이버 developers에서 발급받은 client_secret
    
    #지역검색 url
    url = "https://openapi.naver.com/v1/search/local.json"
    option = "&display=1&start=1&sort=random" # sort: random(유사도 순), comment(카페/블로그 리뷰 개수 순)
    keyword = "하이미디어 종로"
    q_keyword = urllib.parse.quote(keyword)
    query = "?query="+q_keyword
    url_query = url + query + option
    
    #Open API 검색 요청 개체 설정
    request = urllib.request.Request(url_query)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    
    #검색 요청 및 처리
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode == 200):
        response_body = response.read()
        content = response_body.decode('utf-8')
        words = content.replace("\n\t\t\t", '').split(',') # 내용을 ,로 분류
        words_index = [i for i in range(len(words)) if 'roadAddress' in words[i]][0] # 도로명 주소에 해당하는 인덱스 검색
        word = words[words_index].replace('"','') # 내용에서 도로명 주소에 해당하는 키와 값 가져오기
        roadaddr = word.strip('roadAddress:')
        search_addr = [keyword, roadaddr]
        return search_addr
    else:
        return None

In [5]:
def road_addr(): # 위도 경도 구하기
    # kakao api에서 키워드 검색을 통해 주소와 위도 경도를 찾을 수 있으나
    # 키워드와 관련된 장소가 모두 검색되어 위도 경도를 분류하는 작업을 1번 더 해야 하므로
    # 도로명 주소를 입력받아 위도 경도를 구하도록 한다
    
    import json
    import requests
    import math
    from bs4 import BeautifulSoup as bs
    
    # 도로명 주소 함수 호출
    s_addr =  search_road_addr()
    
    # 키워드 추출
    keyword = s_addr[0]
    
    # 도로명 주소 구하기
    addr = s_addr[1]
    
    # 카카오 map api 주소?query=도로명 주소
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+addr
    
    # api 신청 시 발급받은 RestAPI 키
    API_key = 'RestAPI_Key' # kakao developers에서 발급받은 RestAPI Key
    headers = {"Authorization": "KakaoAK " + API_key}
    
    # api를 통해 가져온 결과
    result = json.loads(str(requests.get(url,headers=headers).text))
    
    # 결과에서 도로명 주소가 담긴 인덱스 값 추출
    road_addr = result['documents'][0]['road_address']
    
    # road_addr['x']는 소수점을 포함한 문자열이므로 숫자로 변환하기 위해서는 int가 아닌 float로 변환해야 한다
    # latitude: 위도
    # longitude: 경도
    latitude = math.trunc(float(road_addr['x']))
    longitude = math.trunc(float(road_addr['y']))
    
    # 검색 결과를 리스트에 담아줌
    keyword_result = [keyword, addr, latitude, longitude]
    
    return keyword_result

In [6]:
def find_out_weather(): # 날씨 예측
    # 라이브러리 추가
    import requests
    import json
    import datetime
    
    # 키워드를 통한 위도 경도 구하기
    results = road_addr()
    nx = results[2]
    ny = results[3]
    
    # 공공데이터포털 기상청 단기예보 api 사용
    vilage_weather_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?"
    service_key = "Service_key" # 공공데이터 포털에서 발급받은 service_key
    
    # 날짜, 시간
    today = datetime.datetime.today()
    base_date = today.strftime("%Y%m%d")
    base_time = "0500"
    
    payload = "serviceKey=" + service_key + "&" + "dataType=json" + "&" + "base_date=" + base_date + "&" + "base_time=" + base_time + "&" + "nx=" + str(nx) + "&" + "ny=" + str(ny)
    
    data = dict()
    data['date'] = base_date
    
    weather_data = dict()
    
    res = requests.get(vilage_weather_url + payload)
    items = res.json().get('response').get('body').get('items')
    
    for item in items['item']:
        
        # 기온
        if item['category'] == 'TMP':
            weather_data['tmp'] = item['fcstValue']

        # 기상상태
        if item['category'] == 'PTY':

            weather_code = item['fcstValue']

            if weather_code == '1':
                weather_state = '비'
            elif weather_code == '2':
                weather_state = '비/눈'
            elif weather_code == '3':
                weather_state = '눈'
            elif weather_code == '4':
                weather_state = '소나기'
            else:
                weather_state = '기상 이상 없음'

    weather_data['code'] = weather_code
    weather_data['state'] = weather_state
    
    data['weather'] = weather_data
    
    weather_date = data['date']
    weather_tmp = data['weather']['tmp']
    weather_state = data['weather']['state'] 
    
    # 기상 상태 반환
    return weather_date, weather_tmp, weather_state

In [7]:
def score(img): # 의류 추천에 사용되는 점수 계산
    
    # cloud, weather 선언
    cloud = classify_cloud(img)
    weather = find_out_weather()
    
    # 구름 유형 점수
    if '권운' in cloud:
        c_score = 0
    elif '고적운' in cloud:
        c_score = 0
    elif '적운' in cloud:
        c_score = 0
    elif '권적운' in cloud:
        c_score = 1
    elif '층운' in cloud:
        c_score = 1
    elif '고층운' in cloud:
        c_score = 2
    elif '난층운' in cloud:
        c_score = 2
    elif '층적운' in cloud:
        c_score = 2
    elif '적란운' in cloud:
        c_score = 2
    elif '권층운' in cloud:
        c_score = 3
    else:
        pass
    
    # 날씨 점수
    if '기상 이상 없음' in weather[2]:
        w_score = 0
    elif '비' in weather[2]:
        w_score = 1
    elif '눈' in weather[2]:
        w_score = 1
    elif '비/눈' in weather[2]:
        w_score = 1
    elif '소나기' in weather[2]:
        w_score = 1
    else:
        pass
    
    # 총 점수 계산
    score = c_score + w_score
    
    # 총 점수에 따른 기상 상태, 추천 의류 리스트 구하기
    if score == 0:
        score = '맑아요'
        cloth = ['pants', 'shorts', 'skirt']
    elif score == 1:
        score = '흐려요'
        cloth = ['blouse', 'dress', 'pants', 'shorts', 'skirt', 'Tshirt']
    elif score == 2:
        score = '비나 눈이 올 수 있어요. 우산을 챙겨 가세요'
        cloth = ['jacket', 'pants', 'Tshirt']
    elif score == 3:
        score = '비/눈이 많이 와요. 미끄러지지 않게 조심하세요'
        cloth = ['jacket', 'pants', 'sweater']
    else:
        score = '태풍이 와요.. 이런 날씨엔 집에 있어요'
        cloth = ['pants', 'sweater']
    chat_bot = "runway: {}".format(score)
    Output.insert(END,chat_bot + '\n')
    # 총 점수에 따른 기상 상태, 추천 의류 리스트 반환
    return score, cloth

In [8]:
def recommend_cloth(inputs): # 3개의 랜덤한 의류 추천
    # 라이브러리 추가
    import os
    from PIL import Image
    import random
    
    # 저장된 의류 이미지가 위치한 경로 지정
    path2 = 'C:/Users/hi/Desktop/DeepLearning/img-fashion/' # 의류 이미지 위치 폴더 경로
    fashion = os.listdir(path2)
    fashion_list = [file for file in fashion if file.endswith('.jpg')]
    types = inputs
    kind = [s for s in fashion_list if types in s]
    random_img = random.sample(kind, 3)
    rand_img1 = Image.open(path2+random_img[0])
    rand_img2 = Image.open(path2+random_img[1])
    rand_img3 = Image.open(path2+random_img[2])
    rand_img1 = rand_img1.resize((300, 400))
    rand_img2 = rand_img2.resize((300, 400))
    rand_img3 = rand_img3.resize((300, 400))
    rand_img1_size = rand_img1.size
    rand_img2_size = rand_img2.size
    rand_img3_size = rand_img3.size
    new_image = Image.new('RGB',(3*rand_img1_size[0], rand_img1_size[1]), (250,250,250))
    new_image.paste(rand_img1,(0,0))
    new_image.paste(rand_img2,(rand_img1_size[0],0))
    new_image.paste(rand_img3,(2*rand_img1_size[0],0))
    new_image.show() # 새 창에서 이미지 띄우기
    return new_image

In [9]:
def rand_cloth(img):
    import time, os
    reset_dir = os.chdir('C:/Users/hi/Desktop/DeepLearning/data/chatbot_dataset/cloud_data') # 분석하고자 하는 사진이 위치한 폴더 경로
    score_list = score(img)
    chat_bot= "runway: 오늘 날씨는 " + score_list[0]
    cloth_list = score_list[1]
    for rc_cloth in cloth_list:
        new_image = recommend_cloth(rc_cloth)
        save_dir = os.chdir('C:/Users/hi/Desktop/DeepLearning/recommend_cloth') # 이미지 저장 폴더 경로
        new_image.save('rc_cloth ('+ rc_cloth + ').png', 'PNG') # 폴더에 이미지 저장
    chat_bot= "runway: 총 " + str(len(cloth_list))+ "개의 사진이 저장되었습니다."
    Output.insert(END,chat_bot + '\n')

In [10]:
window = Tk()

window.title('런웨이와 챗봇상담')
window.geometry("850x900+0+100")

## -----------------런웨이 말하는 부분
path="C:/Users/hi/Desktop/DeepLearning/image/runway.png"
if path == '':exit()
image=Image.open(path)
height=50
width=70
canvas=Canvas(window, height=500, width=500)
image=image.resize((height,width),PIL.Image.ANTIALIAS)
photo=ImageTk.PhotoImage(image)
label2=Label(window, image=photo) #라벨 생성, 라벨에는 앞서 선언한 이미지가 들어감.
label2.pack()  #runway image

## -----------------채팅 내용칸
frame2 = Frame(window) 
frame2.pack(side=tk.TOP)
Output = Text(frame2, height = 36,width =100,bg = "light cyan")
Output.configure(font=("Courier", 12, "italic"))
Output.grid(column=0, row=0)

## -----------------채팅 입력칸
frame1 = Frame(window)
frame1.pack(side=tk.BOTTOM)
button = Button(frame1, text="", command=rand_cloth('real_img.jpg'))
button.grid(column=0, row=2)

window.mainloop()